In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
scaffold = ""
# gene_name = "XM_013278379.1"
# species = "Papilio_polytes"

In [3]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
#     print(list_of_files_in_genome_folder)
    if "Temp" in list_of_files_in_genome_folder:
        subprocess.run(f'rm -r "{genome_location}/{species}/Temp"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{genome_location}/{species}/Temp")
    return(genome_file)  

In [4]:
def create_query_folder(annotated_species_location,gene_name,species):
    import time


    list_of_files = os.listdir(annotated_species_location)
    if species in list_of_files:
        subprocess.run(f'rm -r "{annotated_species_location}/{species}"', shell = True, stderr = subprocess.DEVNULL)
    time.sleep(2.5)
    os.mkdir(f"{annotated_species_location}/{species}")
    os.mkdir(f"{annotated_species_location}/{species}/Transcripts")
    os.mkdir(f"{annotated_species_location}/{species}/Transcripts/{gene_name}")
    
    

In [6]:
import os
import subprocess
from Bio import SeqIO
# species_gene = ["Vanessa_tameamea,XM_026631919.1"]
species_gene = ["Vanessa_tameamea,XM_026629288.1","Papilio_xuthus,XM_013311370.1","Colias_croceus,XM_045647888.1","Leptidea_sinapis,XM_050817312.1","Pieris_brassicae,XM_045673832.1","Pieris_napi,XM_047667370.1","Pieris_rapae,XM_045633510.1","Zerene_cesonia,XM_038353076.1"]

# gene_name = "XM_013278379.1"
# species = "Papilio_polytes"
annotated_species_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/14.Cryptochrome_2_Exon_Analysis/1.Annotated Species"
try:
    os.mkdir(annotated_species_location)
except:
    pass
for entries in species_gene:

    species,gene_name = entries.split(",")
    print(species,gene_name)

    genome_location = f"/mnt/f/Genomes_2023-12-26"
    genome_file = get_genome_file(genome_location,species)
    
    query_species = species
    create_query_folder(annotated_species_location,gene_name,species)
    list_of_files = os.listdir(f"{genome_location}/{species}")
    # print(list_of_files)
    for file_names in list_of_files:
        if file_names.endswith("_genomic.gff"):
            gff3_file = file_names

    with open(f"{genome_location}/{species}/{gff3_file}") as gff_file_open:
        gff_file_list = gff_file_open.readlines()
    complement = ''
    exon_count = 1
    gene_found = 0
    exon_dictionary = {}
    for line in gff_file_list:
        if gene_name in line:
            if "CDS" in line.split("\t"):

    #             print(line)
                gene_found = 1
                if complement == '':
                    complement = line.split("\t")[6]
                exon_start = line.split("\t")[3]
                exon_stop = line.split("\t")[4]
                scaffold = line.split("\t")[0]
                exon_dictionary[f"Exon_{exon_count}"] = [exon_start,exon_stop]
                exon_count += 1
            if "exon" in line.split("\t") and gene_found == 1:
                break
    print(complement)
    print(exon_dictionary)
    if complement == "+":
        exon_counter = 1
    elif complement == "-":
        exon_counter = len(exon_dictionary)
    else:
        print("Error!!")
        assert False

    # list_of_query_species = os.listdir(query_location)
    # if query_species not in list_of_query_species:
    #     os.mkdir(f"{annotated_species_location}/{query_species}")
    # list_of_folders_here = os.listdir(f"{annotated_species_location}/{query_species}")
    # if "Transcripts" not
    #     os.mkdir(f"{annotated_species_location}/{query_species}/{gene_name}")

    print_out = ''
    print("exon counter", exon_counter)        
    for i in range(1,len(exon_dictionary)+1):

        exon_start, exon_stop = exon_dictionary[f"Exon_{exon_counter}"]
    #     print(exon_start,exon_stop)
        subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{exon_start}-{exon_stop} > "{genome_location}/{species}/Temp/temp_gene.fa"', shell = True)
        exon_sequence_file = SeqIO.parse(f"{genome_location}/{species}/Temp/temp_gene.fa", "fasta")
        sequence = ''

        for sequences in exon_sequence_file:
    #         print(sequences.id)
            if complement == "-":
                sequence = sequences.seq.reverse_complement() 

    #             print(sequences.seq.reverse_complement().translate())
            else:
                sequence = sequences.seq
    #             print(sequences.seq)
    #             print(sequences.seq.translate())
            print_out += str(len(sequence))+"\t"
            output = f">{query_species}_{gene_name}_Exon{exon_counter}\n{sequence}"
        with open(f"{annotated_species_location}/{query_species}/Transcripts/{gene_name}/Exon{exon_counter}.fa", 'w') as outfile:
            outfile.write(output)
        print(output)
        if complement == "-":
            exon_counter -= 1
        else:
            exon_counter += 1

    #         if gene_location in line.split("\t"):
    #           print(line)
    #           break
    print(print_out)

Vanessa_tameamea XM_026629288.1
-
{'Exon_1': ['5026318', '5026559'], 'Exon_2': ['5025671', '5025922'], 'Exon_3': ['5023916', '5024100'], 'Exon_4': ['5022874', '5023045'], 'Exon_5': ['5022270', '5022495'], 'Exon_6': ['5021636', '5021776'], 'Exon_7': ['5020960', '5021111'], 'Exon_8': ['5018310', '5018512'], 'Exon_9': ['5017500', '5018203']}
exon counter 9


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Vanessa_tameamea_XM_026629288.1_Exon9
gtTCATCAATTCCACCGATATTACAAAGGCCAAATGTAACACAATCTTCACCGAGTCCAACATCcataattgcaaatataaatcaatcaaattatttatgtagtcAAACACCTGAATCCGCTTCACAGTCATCTCCGACTACACAAATCAAAGATGACGTTTTCCTCCGACCAACTAACATCCGAAGTGTTGCTACAAATAAAGCAACACCTTTTAAGCATGTTGTAATTGTGGAACAGAATAGGCGAGAGGAAATCGAAACCGAAAATATGAATGGATCAAAGTTTGACGATACAaacaaatcttttaaaattaatgacaacACCGATACTATGCAAAACTCGAAAAAATCGGGAAACTacgatttcaaaaatttaatcataaataattgtatacagAAGTTTTCTAGTGACCAGTTGGCATTTTTAAATCAACGAACCAATAAAAATGACCATTACAGCggtgatattaaaatagatgAATACAGCACAAGCAAAccgaaattatttttcatagatAATGGAGTAATGTCTCATAATGACAGCGAACAGAACTTTACATCTAATAACTACAGAGATAACTATCACGGTAAGGATAGCCATACCAAAGTACAGAGTGCAGAATctaaaaatgaacataaaatgAACACAAATAGTTCAAGTCCATCCGATCCTAAGAATATatcagataatataaataaataa


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Vanessa_tameamea_XM_026629288.1_Exon8
GAAGTATATTCCAGCGTTGAAGAATTTACCAACTCGTTATATTCACGAGCCTTGGATGGCGCCGGAAGCGGTGCAACAAGCTGCGAACTGTATCATTGGACGAGACTACCCACAGCCGATAGTCGACCACGCTAAGGCATCGCAGGTTAACATCGAGCGCATCAAGCAAGTTTATGCCCAACTCGCCAAGTACAAACCTCCAG


...

samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Zerene_cesonia_XM_038353076.1_Exon6
GGTCAAACTGGATTTCCGTGGATTGATGCTATTATGATTCAGCTTAGAGAAGAAGGCTGGATCCACCATTTGGCTAGACATGCTGTTGCTTGTTTCCTAACTCGGGGCGACTTATGGATCTCATGGGAAGAAGGGATGAAG
>Zerene_cesonia_XM_038353076.1_Exon7
GTATTTGATGAGCTACTATTAGATGCAGATTGGTCTGTGAATGCAGGAATGTGGATGTGGTTATCATGTTCTTCcttttttcaacaattttttcacTGCTACTGCCCAGTTAGATTTGGACGTAAAACAGATCCAAATGGAGATTTTATAAG


samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)


>Zerene_cesonia_XM_038353076.1_Exon8
GAAATATATTCCAGCGCTGAAGAACATGCCAACTAGGTATATTCATGAACCATGGGTAGCACCAGATTCCGTACAACAGGCAGCTTGTTGCATAATTGGCCGGGATTATCCACTACCTATGGTGGACCACACAAAGGCGtcacaaattaatattgaacgTATAAAACAAGTTTATGCACAACTGGCTAAATATAAACCACAAG
>Zerene_cesonia_XM_038353076.1_Exon9
aAACTCAATTTTTTGTGATTGCAGGTGTAATAAATCAAGTCATTCAAAGACCAAATATTATGCAATCATCACCTAGTCCCACATCTATAATCACAAGTATTAATCAGTCAAATTACTTGTGTAGTCAAATAACGGAGGTGCAAAATGTGACAACCCCATCCACTCAAGCATTGAAAGAAGATGACATATTCATTCGACCTAATAATGTAGACACAAAACAACCGTTGTTTAAGCAAGTTCTGCTGGTTCAACAACAAAAGGCTTCAAAATTGGTACAGCAATCAAGCTCCTGTACAGCACCCAAAGAGCATTATCTTATGAATGGTGAtctcaatatttacaaaatcgGTAACCCTGAGGATTTGCaaaatgctaataaaattGAGAATTGTCACCATAAAAATCTCACACTGAATAGACGCAacgacaattttaataaaaacctaaaTGAGTTCGAAAAGAATTCAGATACAAGTGGGACTTATGCTCAAAACATCAAAATTGTAGATTATGAAAACGAAAAGCCGAAGTTATTCATGATGGCCAATGGTATTGTGCCGCACAATAATCTAGAACAGAATTACCTAAATTCTTCCTTAAGAAATGAGTATAATTCAGAAgacaaaaaatctaaaagcAAAGGTGGCAAAATTCatcatattgttttaaatgacaGACAGTTTCTTGGGTTGCCAGATAAAAATACTGAGCATAATGAAAAC

samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libtinfow.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
samtools: /home/saurav/miniconda3/envs/ncbi_datasets/bin/../lib/libncursesw.so.6: no version information available (required by samtools)
